In [1]:
import json
import os
from copy import deepcopy
from typing import List, Tuple

In [2]:
hypothesis = 'pts_shallow_encoder_mlps_curated_dataset'

In [3]:
base_configuration = {
    'hypothesis': hypothesis,
    'experiment': None,  # will change below
    'data': {
        'path': '/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/data_preparation/patch_to_score/curate_dataset/curated_folds_training_dicts.pkl',
        'use_pesto': True,
        'ablation_string': '111111111',
        'max_folds': None
    },
    'model': {
        'name': 'pts_transformer_encoder_mlp',
        'kwargs': {
            'hidden_sizes_mlp_a': None,  # will change below
            'mlp_a_dropout_rate': 0.2,
            'hidden_sizes_mlp_c': None,  # will change below
            'mlp_c_dropout_rate': 0.1,
            'activation': 'relu',
            'input_shape': (10, 9),
            'max_number_of_patches': 10
        }
    },
    'compile': {
        'optimizer': {
            'name': 'adamw',
            'kwargs': {'learning_rate': 1e-3}
        },
        'loss': {
            'name': 'binary_cross_entropy',
            'kwargs': {}
        }
    },
    'fit': {
        'epochs': 200,
        'batch_size': 256,
        'verbose': 1,
        'n_early_stopping_epochs': 12
    }
}

In [4]:
def mlps_option_to_str(mlps_option: List[Tuple[int, int]]) -> str:
    res = f'{str(mlps_option[0][0])}-{str(mlps_option[0][1])}'
    return f'{len(mlps_option)}_{res}'

In [5]:
dir_path = f'../data/{hypothesis}'
os.makedirs(dir_path, exist_ok=True)

hidden_sizes_mlp_a_options = [
    [(512, 512)],
    [(1024, 512)],
    [(512, 512) for _ in range(2)]
]

hidden_sizes_mlp_c_options = deepcopy(hidden_sizes_mlp_a_options)

for hidden_sizes_mlp_a in hidden_sizes_mlp_a_options:
    for hidden_sizes_mlp_c in hidden_sizes_mlp_c_options:
        experiment_name = f'hidden_sizes_mlp_a_{mlps_option_to_str(hidden_sizes_mlp_a)}_hidden_sizes_mlp_c_{mlps_option_to_str(hidden_sizes_mlp_c)}'
        configuration = base_configuration.copy()
        configuration['experiment'] = experiment_name
        configuration['model']['kwargs']['hidden_sizes_mlp_a'] = hidden_sizes_mlp_a
        configuration['model']['kwargs']['hidden_sizes_mlp_c'] = hidden_sizes_mlp_c

        with open(f'{dir_path}/{experiment_name}.json', 'w') as f:
            json.dump(configuration, f, indent=4)